In [ ]:
# import openai
# from openai import OpenAI

# client = OpenAI()

# completion = client.chat.completions.create(
#     model="gpt-4o-mini",
#     messages=[
#         {"role": "system", "content": "You are a helpful assistant."},
#         {
#             "role": "user",
#             "content": "Write a haiku about the little mermaid."
#         }
#     ]
# )

# print(completion.choices[0].message)

ChatCompletionMessage(content='Beneath the sea’s waves,  \nA heart yearns for the sunset,  \nSilent love takes flight.', role='assistant', function_call=None, tool_calls=None, refusal=None)


In [25]:
import openai
import json
import datetime
import pytz

client = OpenAI()
# 2. Giả lập các hàm

# Hàm lấy thời tiết
def get_weather(city):
    sample_weather_data = {
        "Hà Nội": {"temperature": "30°C", "description": "Nắng nhẹ"},
        "Hồ Chí Minh": {"temperature": "32°C", "description": "Nắng nóng"},
        "Đà Nẵng": {"temperature": "28°C", "description": "Mưa rào"}
    }
    return sample_weather_data.get(city, {"temperature": "N/A", "description": "Không tìm thấy dữ liệu"})

# Hàm lấy thời gian
def get_time(city):
    city_timezones = {
        "Hà Nội": "Asia/Ho_Chi_Minh",
        "Hồ Chí Minh": "Asia/Ho_Chi_Minh",
        "New York": "America/New_York",
        "London": "Europe/London"
    }
    timezone = city_timezones.get(city)
    if timezone:
        city_time = datetime.datetime.now(pytz.timezone(timezone))
        return {"time": city_time.strftime("%Y-%m-%d %H:%M:%S")}
    else:
        return {"time": "Không tìm thấy múi giờ cho thành phố này"}

# Hàm lấy tỷ giá ngoại tệ
def get_exchange_rate(currency_pair):
    sample_exchange_data = {
        "USD/VND": 24000,
        "EUR/VND": 26000,
        "USD/EUR": 0.92
    }
    return {"rate": sample_exchange_data.get(currency_pair, "Không tìm thấy tỷ giá cho cặp tiền này")}

# 3. Xử lý Function Calling
def chat_with_gpt(user_input):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Bạn là một trợ lý AI có thể cung cấp thông tin thời tiết, thời gian và tỷ giá ngoại tệ."},
            {"role": "user", "content": user_input}
        ],
        functions=[
            {
                "name": "get_weather",
                "description": "Lấy thông tin thời tiết của một thành phố.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "city": {
                            "type": "string",
                            "description": "Tên thành phố"
                        }
                    },
                    "required": ["city"]
                }
            },
            {
                "name": "get_time",
                "description": "Lấy thời gian hiện tại của một thành phố.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "city": {
                            "type": "string",
                            "description": "Tên thành phố"
                        }
                    },
                    "required": ["city"]
                }
            },
            {
                "name": "get_exchange_rate",
                "description": "Lấy tỷ giá ngoại tệ.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "currency_pair": {
                            "type": "string",
                            "description": "Cặp tiền tệ, ví dụ: USD/VND"
                        }
                    },
                    "required": ["currency_pair"]
                }
            }
        ],
        function_call="auto"
    )
    
    print (response)
    response_message = response.choices[0].message

    return response_message
    

In [26]:
user_input = "Kể cho tôi một câu chuyện cười"
response_message = chat_with_gpt(user_input)
print (response_message)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: YOUR_API_KEY. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [17]:
if response_message.function_call:
    function_name = response_message.function_call.name
    arguments = json.loads(response_message.function_call.arguments)
    
    if function_name == "get_weather":
        city = arguments.get("city")
        weather_info = get_weather(city)
        result =  f"Thời tiết ở {city}: {weather_info['temperature']}, {weather_info['description']}."
    
    if function_name == "get_time":
        city = arguments.get("city")
        time_info = get_time(city)
        result =  f"Thời gian hiện tại ở {city}: {time_info['time']}."
    
    if function_name == "get_exchange_rate":
        currency_pair = arguments.get("currency_pair")
        rate_info = get_exchange_rate(currency_pair)
        result =  f"Tỷ giá {currency_pair}: {rate_info['rate']}."

else:
    result =  response_message.content

In [18]:
print(f"Trợ lý: {result}")

Trợ lý: Có một anh chàng kia rất thích đánh golf nhưng lại không mấy khéo tay. Một hôm, anh ta đi chơi golf với bạn bè, và khi vừa đánh một cú swing, bóng golf bay thẳng vào mặt một bà cụ đang đứng gần đó. Bà cụ tức giận gọi anh ta lại và nói:

- Cậu có biết là cậu phải xin lỗi không?

Anh chàng bối rối nói:

- Xin lỗi! Nhưng làm ơn cho tôi biết, bà có cần mắt không?

Bà cụ hỏi lại:

- Tại sao cậu lại hỏi như vậy?

Anh chàng cười và đáp:

- Bởi vì nếu bà không cần mắt, tôi sẽ tặng bà một cặp kính mới!


In [37]:
import openai
from openai import OpenAI
import json

client = OpenAI()

# Hàm tắt đèn
def turn_off_light():
    # global led_status

    # cmd_queue.put('0')  # Gửi lệnh tắt LED
    # led_status = 'OFF'
    # update_ui()
    return 0

# Hàm bật đèn
def turn_on_light():
    # global led_status

    # cmd_queue.put('1')  # Gửi lệnh bật LED
    # led_status = 'ON'
    # update_ui()
    return 0

# Hàm trả về trạng thái đèn
def get_led_status():
    global led_status

    if led_status == 'ON':
        return "Đèn đang bật"
    elif led_status == 'OFF':
        return "Đèn đang tắt"
    elif led_status == 'BLINKING':
        return "Đèn đang nhấp nháy"
    else:
        return "Không xác định được trạng thái đèn"

# Xử lý Function Calling với GPT
def chat_with_gpt(user_input):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Bạn là một trợ lý AI giúp điều khiển đèn LED IoT. Bạn có thể bật, tắt và kiểm tra trạng thái đèn LED. Nếu trời sáng quá, hãy tắt đèn, còn nếu trời tối, hãy bật đèn lên nhé!"},
            {"role": "user", "content": user_input}
        ],
        functions=[
            {
                "name": "turn_on_light",
                "description": "Bật đèn LED."
            },
            {
                "name": "turn_off_light",
                "description": "Tắt đèn LED."
            },
            {
                "name": "get_led_status",
                "description": "Lấy trạng thái hiện tại của đèn LED."
            }
        ],
        function_call="auto"
    )
    
    response_message = response.choices[0].message
    
    if response_message.function_call:
        function_name = response_message.function_call.name
        
        if function_name == "turn_on_light":
            return turn_on_light()
        
        if function_name == "turn_off_light":
            return turn_off_light()
        
        if function_name == "get_led_status":
            return get_led_status()
    
    else:
        return response_message.content

        

In [47]:
user_input = "It's so dark in here"
result = chat_with_gpt(user_input)


In [48]:
print(f"Trợ lý: {result}")

Trợ lý: {'status': 'LED đã bật'}
